In [89]:
import pandas as pd
import numpy as np
from geopy.geocoders import MapBox
from geopy.extra.rate_limiter import RateLimiter
from keys import my_key 
import requests
import xml.etree.ElementTree as et

In [57]:
pf_df = pd.read_excel("data_sources/pf_locations.xlsm",sheet_name='pf_locations')

pf_df.head()

,Street,City,State,Link,Full Address
0,1 Foster Ave,Crafton,PA,https://www.planetfitness.com/gyms/crafton-pa,"1 Foster Ave, Crafton, PA"
1,1 Route 37 West,Toms River,NJ,https://www.planetfitness.com/gyms/toms-river-...,"1 Route 37 West, Toms River, NJ"
2,1 York Gate Boulevard,North York,ON,https://www.planetfitness.com/gyms/north-york-...,"1 York Gate Boulevard, North York, ON"
3,10 28th Ave SW,Minot,ND,https://www.planetfitness.com/gyms/minot-nd,"10 28th Ave SW, Minot, ND"
4,10 East St,East Granby,CT,https://www.planetfitness.com/gyms/east-granby-ct,"10 East St, East Granby, CT"


In [58]:
geolocator = MapBox(api_key=my_key)

geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
pf_df['geo_data'] = pf_df['Full Address'].apply(geocode)

pf_df.head()

,Street,City,State,Link,Full Address,geo_data
0,1 Foster Ave,Crafton,PA,https://www.planetfitness.com/gyms/crafton-pa,"1 Foster Ave, Crafton, PA","(1 Foster Avenue, Crafton, Pennsylvania 15205,..."
1,1 Route 37 West,Toms River,NJ,https://www.planetfitness.com/gyms/toms-river-...,"1 Route 37 West, Toms River, NJ","(1 Little League World Champions Boulevard, To..."
2,1 York Gate Boulevard,North York,ON,https://www.planetfitness.com/gyms/north-york-...,"1 York Gate Boulevard, North York, ON","(1 York Gate Boulevard, North York, Ontario M3..."
3,10 28th Ave SW,Minot,ND,https://www.planetfitness.com/gyms/minot-nd,"10 28th Ave SW, Minot, ND","(10 28th Avenue Southwest, Minot, North Dakota..."
4,10 East St,East Granby,CT,https://www.planetfitness.com/gyms/east-granby-ct,"10 East St, East Granby, CT","(10 East Street, East Granby, Connecticut 0602..."


In [82]:
print(pf_df.geo_data[1].address)

1 Little League World Champions Boulevard, Toms River, New Jersey 08753, United States


In [84]:
lat = []
long = []
address = []

for loc in pf_df.geo_data:
    lat.append(loc.latitude)
    long.append(loc.longitude)
    address.append(loc.address)

pf_df["latitude"] = lat
pf_df["longitude"] = long
pf_df["address"] = address


pf_df.head()

,Street,City,State,Link,Full Address,geo_data,latitude,longitude,address
0,1 Foster Ave,Crafton,PA,https://www.planetfitness.com/gyms/crafton-pa,"1 Foster Ave, Crafton, PA","(1 Foster Avenue, Crafton, Pennsylvania 15205,...",40.441312,-80.069116,"1 Foster Avenue, Crafton, Pennsylvania 15205, ..."
1,1 Route 37 West,Toms River,NJ,https://www.planetfitness.com/gyms/toms-river-...,"1 Route 37 West, Toms River, NJ","(1 Little League World Champions Boulevard, To...",39.965006,-74.203726,"1 Little League World Champions Boulevard, Tom..."
2,1 York Gate Boulevard,North York,ON,https://www.planetfitness.com/gyms/north-york-...,"1 York Gate Boulevard, North York, ON","(1 York Gate Boulevard, North York, Ontario M3...",43.758503,-79.519441,"1 York Gate Boulevard, North York, Ontario M3N..."
3,10 28th Ave SW,Minot,ND,https://www.planetfitness.com/gyms/minot-nd,"10 28th Ave SW, Minot, ND","(10 28th Avenue Southwest, Minot, North Dakota...",48.204934,-101.294666,"10 28th Avenue Southwest, Minot, North Dakota ..."
4,10 East St,East Granby,CT,https://www.planetfitness.com/gyms/east-granby-ct,"10 East St, East Granby, CT","(10 East Street, East Granby, Connecticut 0602...",41.939280,-72.716430,"10 East Street, East Granby, Connecticut 06026..."


In [85]:
# Store dataframe before continuing so I don't overuse my free account.
pf_df.to_csv('pf_post_mapbox.csv')

In [111]:
ski_df = pd.read_excel('data_sources/ski_resorts.xlsx')

ski_df.head()


,Location Type,Special Features,Location Name,Lng,Lat
0,Ski Pass Map,Epic,Stevens Pass,-121.089020,47.744812
1,Ski Pass Map,Epic,Northstar California Resort,-120.121093,39.274568
2,Ski Pass Map,Epic,Heavenly Mountain Resort,-119.942842,38.956924
3,Ski Pass Map,Epic,Kirkwood Mountain Resort,-120.065167,38.684751
4,Ski Pass Map,Epic,Park City Mountain,-111.507995,40.651420


In [120]:
coordinates = ski_df.Lat.astype(str) + ', ' + ski_df.Lng.astype(str)

coordinates

0      47.7448119, -121.0890197
1      39.2745678, -120.1210934
2      38.9569241, -119.9428424
3      38.6847514, -120.0651665
4      40.6514199, -111.5079947
                 ...           
434     39.3601951, -106.301445
435    39.3997821, -107.3387647
436      42.9200364, -88.016061
437       43.601705, -72.180265
438     41.5175274, -71.5272242
Length: 439, dtype: object

In [121]:
reverse = RateLimiter(geolocator.reverse, min_delay_seconds=1)

addresses =[]

addresses = coordinates.apply(reverse)

ski_df['address'] = [x.address for x in addresses]

,Location Type,Special Features,Location Name,Lng,Lat,address
0,Ski Pass Map,Epic,Stevens Pass,-121.089020,47.744812,"(Fall Line Sports, 93001 Summit US Hwy 2, Skyk..."
1,Ski Pass Map,Epic,Northstar California Resort,-120.121093,39.274568,"(Overlook Bar at Northstar, Truckee, Californi..."
2,Ski Pass Map,Epic,Heavenly Mountain Resort,-119.942842,38.956924,"(Heavenly Sports -Tamarack, 4080 Lake Tahoe Bl..."
3,Ski Pass Map,Epic,Kirkwood Mountain Resort,-120.065167,38.684751,"(1501 Kirkwood Meadows Drive, Kirkwood, Califo..."
4,Ski Pass Map,Epic,Park City Mountain,-111.507995,40.651420,"(Miner's Camp, 1345 Lowell Ave, Miner's Camp L..."


In [124]:
ski_df['address'] = [x.address for x in ski_df['address']]

In [125]:
ski_df.head()

,Location Type,Special Features,Location Name,Lng,Lat,address
0,Ski Pass Map,Epic,Stevens Pass,-121.089020,47.744812,"Fall Line Sports, 93001 Summit US Hwy 2, Skyko..."
1,Ski Pass Map,Epic,Northstar California Resort,-120.121093,39.274568,"Overlook Bar at Northstar, Truckee, California..."
2,Ski Pass Map,Epic,Heavenly Mountain Resort,-119.942842,38.956924,"Heavenly Sports -Tamarack, 4080 Lake Tahoe Blv..."
3,Ski Pass Map,Epic,Kirkwood Mountain Resort,-120.065167,38.684751,"1501 Kirkwood Meadows Drive, Kirkwood, Califor..."
4,Ski Pass Map,Epic,Park City Mountain,-111.507995,40.651420,"Miner's Camp, 1345 Lowell Ave, Miner's Camp Lo..."


In [132]:
pf_df.rename(columns={'latitude':'Lat', 'longitude': 'Lng'},inplace=True)
pf_df.drop(columns=['Street', 'City', 'State', 'geo_data', 'Full Address'],inplace=True)

pf_df['Location Type'] = 'Gym'
pf_df['Location Name'] = 'Planet Fitness'

In [133]:
pf_df.head()

,Link,Lat,Lng,address,Location Type,Location Name
0,https://www.planetfitness.com/gyms/crafton-pa,40.441312,-80.069116,"1 Foster Avenue, Crafton, Pennsylvania 15205, ...",Gym,Planet Fitness
1,https://www.planetfitness.com/gyms/toms-river-...,39.965006,-74.203726,"1 Little League World Champions Boulevard, Tom...",Gym,Planet Fitness
2,https://www.planetfitness.com/gyms/north-york-...,43.758503,-79.519441,"1 York Gate Boulevard, North York, Ontario M3N...",Gym,Planet Fitness
3,https://www.planetfitness.com/gyms/minot-nd,48.204934,-101.294666,"10 28th Avenue Southwest, Minot, North Dakota ...",Gym,Planet Fitness
4,https://www.planetfitness.com/gyms/east-granby-ct,41.939280,-72.716430,"10 East Street, East Granby, Connecticut 06026...",Gym,Planet Fitness


In [134]:
locations_df = pd.concat([pf_df,ski_df])

locations_df.head()

,Link,Lat,Lng,address,Location Type,Location Name,Special Features
0,https://www.planetfitness.com/gyms/crafton-pa,40.441312,-80.069116,"1 Foster Avenue, Crafton, Pennsylvania 15205, ...",Gym,Planet Fitness,NaN
1,https://www.planetfitness.com/gyms/toms-river-...,39.965006,-74.203726,"1 Little League World Champions Boulevard, Tom...",Gym,Planet Fitness,NaN
2,https://www.planetfitness.com/gyms/north-york-...,43.758503,-79.519441,"1 York Gate Boulevard, North York, Ontario M3N...",Gym,Planet Fitness,NaN
3,https://www.planetfitness.com/gyms/minot-nd,48.204934,-101.294666,"10 28th Avenue Southwest, Minot, North Dakota ...",Gym,Planet Fitness,NaN
4,https://www.planetfitness.com/gyms/east-granby-ct,41.939280,-72.716430,"10 East Street, East Granby, Connecticut 06026...",Gym,Planet Fitness,NaN


In [136]:
locations_df.to_csv('data_sources/combined_locations.csv')